# Custom Tutorial 1a: Creating a custom model version of the UGM_flip model

If you have not done so already, install PyBEAM. Go to the PyBEAM github (https://github.com/murrowma/pybeam) for instructions. Then, go through Custom Tutorial 1 to learn how to build the simple DDM.

Once PyBEAM is installed, we can create our first model. First, download the folder "custom_model_template.zip" and unzip it. Then, copy the files to a new directory.

To do this, create a new folder anywhere on your device and copy the files located in folder "custom_model_template" to it. There will be three files you need to copy: "setup.py", "functions.pyx", and "model.pyx". The only file that needs to be modified is "model.pyx". DO NOT TOUCH THE OTHER TWO FILES.

In this file, we create the UGM_flip model contained in the PyBEAM publication and PyBEAM's precoded model set. It has the following 8 parameters:

    'tnd': non-decision time
    'w': relative start point
    'mu': drift rate
    'l': leakage rate
    'k': urgency parameter
    'sigma': scaling parameter
    't_flip': time when mu changes sign
    'b': decision threshold location (upper is at b, lower is
         at -b; separation is 2b)
         
This gives us the following custom model.pyx file:


As an example, lets recreate another model we have already seem from earlier PyBEAM tutorials: UGM_flip. Doing so produces the following model.pyx file:

    """
    # Import Python packages and C libraries.
    
    """

    import cython
    cimport cython

    from libc.math cimport pow, exp, log, sin, cos

    """
    # Model parameter functions.
    
    """

    # total number of input parameters
    DEF N_phi = 8

    # function for the non-decision time
    cdef double non_decision_time(double phi[N_phi]):
        return phi[0]

    # function for the relative start point
    cdef double relative_start(double phi[N_phi]):
        return phi[1]

    # function for the drift rate
    cdef double drift(double phi[N_phi], double x, double t):
        cdef double mu = phi[2]
        cdef double l = phi[3]
        cdef double k = phi[4]
        cdef double t_flip = phi[5]

        if (t < t_flip):
            return mu*(1.0 + k*t) + ( k/(1.0 + k*t) - l )*x
        else:
            return -mu*(1.0 + k*t) + ( k/(1.0 + k*t) - l )*x

    # function for the diffusion rate
    cdef double diffusion(double phi[N_phi], double x, double t):
        cdef double sigma = phi[6]
        cdef double k = phi[4]

        return sigma*(1.0 + k*t)

    # function for the upper decision threshold
    cdef double upper_decision_threshold(double phi[N_phi], double t):
        return phi[7]

    # function for the lower decision threshold
    cdef double lower_decision_threshold(double phi[N_phi], double t):
        return -upper_decision_threshold(phi, t)

    # function for the contamination strength
    cdef double contamination_strength(double phi[N_phi]):
        return 0.0

    # function for the contamination probability
    cdef double contamination_probability(double phi[N_phi], double t):
        return 0.0

    """
    # Function to modify time step with unusual likelihood functions.
    
    """

    # function to modify the time step
    cdef double modify_dt(double phi[N_phi], double t):
        return 1.0


Once you have finished the model.pyx file, you now need to compile the function. To do so, open terminal/anaconda prompt/command prompt (depending on OS), and navigate to the folder containing your model files. Then, run the following line of code:
    
    python setup.py build_ext --inplace
    
And with that, your model is ready to be used with PyBEAM!
